## 1. Import Packages

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
import re
import glob
import shutil
from PIL import Image

## 2. 데이터셋 다운로드 및 훈련, 검증, 테스트 데이터셋 구분
이번 실습에서는 고양이와 강아지 이미지를 분류하는 네트워크를 학습시킬 것입니다. 
우리가 이번에 사용할 데이터셋은 2013년 후반에 캐글에서 컴퓨터 비전 경연 대회의 일환으로 제작되었습니다. 참고로 CNN을 사용한 참가자가 95% 정확도를 달성하여 당시 대회 우승을 차지했습니다. 

강아지-고양이 이미지 데이터셋은 [여기](https://www.kaggle.com/c/dogs-vs-cats/data)에서 다운로드 받을 수 있습니다. 다운로드를 위해서는 캐글 계정이 필요한데, 많은 시간이 소요되지는 않을 겁니다 (페이스북 및 구글 계정으로 로그인 가능).
데이터를 '../data' 경로에 다운받으신 후 모든 압축을 해제하시기 바랍니다. 

다운받으신 원본 데이터셋에는 고양이와 강아지 이미지가 각각 12500개로 총 25000개의 학습용 데이터셋이 구성되어 있습니다. 
이 정도면 강아지와 고양이를 학습시키에는 어느정도 충분한 양입니다. 하지만 이번 실습에서 우리는 훨씬 더 적은 양의 데이터만 사용해 볼 것입니다.
아래의 코드를 실행하면 '../data/my_cat_dog' 경로에 train, validation, test 데이터셋이 구분되어 생성이 됩니다.
훈련 데이터셋은 class당 1000개, 검증은 class당 500개, 마지막으로 테스트는 class당 1000개로 구성되어 있습니다.  




In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

data_dir = '../data/dogs-vs-cats'
output_dir = '../data/my_cat_dog'
all_cat = glob.glob(os.path.join(data_dir, 'train', 'cat*'))
all_dog = glob.glob(os.path.join(data_dir, 'train', 'dog*'))
all_cat.sort(key=natural_keys)
all_dog.sort(key=natural_keys)

train_cat = all_cat[:1000]
train_dog = all_dog[:1000]
val_cat = all_cat[1000:1500]
val_dog = all_dog[1000:1500]
test_cat = all_cat[1500:2500]
test_dog = all_dog[1500:2500]

train_cat_dir = os.path.join(output_dir, 'train', 'cat')
train_dog_dir = os.path.join(output_dir, 'train', 'dog')
val_cat_dir = os.path.join(output_dir, 'val', 'cat')
val_dog_dir = os.path.join(output_dir, 'val', 'dog')
test_cat_dir = os.path.join(output_dir, 'test', 'cat')
test_dog_dir = os.path.join(output_dir, 'test', 'dog')

os.makedirs(output_dir, exist_ok=True)
os.makedirs(train_cat_dir, exist_ok=True)
os.makedirs(train_dog_dir, exist_ok=True)
os.makedirs(val_cat_dir, exist_ok=True)
os.makedirs(val_dog_dir, exist_ok=True)
os.makedirs(test_cat_dir, exist_ok=True)
os.makedirs(test_dog_dir, exist_ok=True)

for c in train_cat:
    shutil.copy(c, train_cat_dir)
for d in train_dog:
    shutil.copy(d, train_dog_dir)
for c in val_cat:
    shutil.copy(c, val_cat_dir)
for d in val_dog:
    shutil.copy(d, val_dog_dir)
for c in test_cat:
    shutil.copy(c, test_cat_dir)
for d in test_dog:
    shutil.copy(d, test_dog_dir)

## 3. 하이퍼파라미터 세팅
학습에 필요한 하이퍼파리미터의 값을 초기화해줍니다.

미니배치의 크기, 학습할 Epoch(세대) 수, Learning rate(학습률) 값들을 다음과 같이 정합니다. 

In [ ]:
batch_size = 100
num_epochs = 10
learning_rate = 0.0001

## 4. Dataset 및 DataLoader 할당
이제 우리가 사용할 데이터셋을 정의해야 합니다. 이전 실습에서는 파이토치에 이미 정의되어 있는 FashionMNIST Dataset class를 불러와서 사용하기만 하면 됐습니다. 따라서 따로 Dataset class를 구현해야 할 필요가 없었습니다. 

이번 실습에서 우리는 우리가 가진 데이터셋에 대한 Dataset class를 정의할 것입니다. Dataset class를 customize 하기 위해서는 [torch.utils.data.Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) class를 상속하고, **\__getitem__** 함수와 **\__len__**함수를 구현해야 합니다. 

**\__getitem__**함수는 어떠한 인덱스를 인자로 받으면, 그에 상응하는 데이터를 반환하는 함수입니다. 
**\__len__**함수는 정의하고자 하는 데이터셋의 총 데이터 개수를 반환하는 함수입니다. 

생성자(**\__init__**)는 아래에 미리 구현을 해두었습니다. 
- **self.all_data**는 정의하는 데이터셋에 포함된 모든 데이터 파일의 경로를 저장하고 있습니다. 인자로 받은 **mode**('train' 또는 'val' 또는 'test')에 따라 훈련용 혹은 검증용 혹은 테스트용 데이터셋을 구분합니다.
- **self.transform**는 [transforms.Compose()](https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=compose#torchvision.transforms.Compose)를 통해 구성된 데이터 전처리 모듈입니다. 

이제 다음을 읽고 여러분 직접 **\__getitem__ 함수**를 완성해보세요.
- 인자로 받은 **index** 변수를 이용하여 그 인덱스에 해당하는 데이터 경로를 정의하세요.
- PIL패키지의 Image.open을 통해 위에서 지정한 데이터 경로에 있는 이미지를 PIL 이미지 객체로 변환하고, 그 결과를 **img** 라는 이름의 변수에 저장하세요. 
- 만일 **self.transform**에 정의된 것이 있다면, 즉 None이 아니라면, **img**에 self.transform을 통해 데이터 전처리를 적용하세요. 그 결과는 다시 **img**변수에 저장되어야 합니다. 
- 마지막으로 데이터의 label을 정의할 차례입니다. 만약 반환하는 이미지가 고양이라면 **label** 변수에 0을, 강아지라면 1을 저장하세요. 데이터 파일의 파일 이름을 통해 이미지가 고양이인지 강아지인지 알 수 있습니다. (힌트: 사람에 따라 다양한 방식으로 구현할 수 있겠지만, [os.path.basename](https://docs.python.org/3/library/os.path.html) 함수를 이용하면 파일 경로에서 파일 이름(+확장자)만 분리할 수 있습니다. 그리고 [str.startswith](https://www.tutorialspoint.com/python/string_startswith.htm)을 활용하면 파일명이 cat으로 시작하는지 혹은 dog으로 시작하는지 알 수 있습니다.)

다음을 읽고 **\__len__ 함수**를 완성해보세요.
- 이 함수는 매우 간단합니다. 정의할 데이터셋의 총 데이터 개수를 반환하세요.

In [ ]:
class CatDogDataset(data.Dataset):
    def __init__(self, data_dir, mode, transform=None):
        self.all_data = glob.glob(os.path.join(data_dir, mode, '*', '*'))
        self.transform = transform
    
    def __getitem__(self, index):
        # 코드 시작
        
        # 코드 종료
        return img, label
    
    def __len__(self):
        # 코드 시작
        
        # 코드 종료

우리는 이번 실습에서 모델을 학습시키는 데에 비교적 적은 양의 데이터셋을 사용하고 있습니다. 이처럼 적은 양의 훈련 데이터를 통해 학습시킨 모델은 오버피팅의 문제가 매우 심각할 수 있습니다. 데이터 증식(data augmentation) 기법은 이러한 작은 데이터셋의 한계를 어느정도 극복하기 위한 좋은 방법입니다. 데이터 증식은 기존의 데이터에 약간의 변형을 가해 데이터의 총량을 증식시키는 효과를 주는 기법을 말합니다.

데이터 증식을 통한 학습과정을 도식화하면 다음과 같습니다. 
<img src="../img/data_aug.png" width="60%" height="60%">

데이터 증식 기법에는 여러가지가 있습니다. 우리는 모델 훈련과정에서 다양한 증식 기법을 랜덤하게 적용하여 훈련용 데이터를 증식하는 효과를 얻을 것입니다.

아래 코드의 **data_transforms** 딕셔너리에는 훈련용, 검증용(또는 테스트용) transforms 모듈이 정의되어 있습니다. 훈련용 transforms 모듈에서는 총 3가지의 데이터 증식 기법과, 이미지를 0~1 사이의 값으로 정규화하고 Pytorch Tensor로 변환하는 [ToTensor()](https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=totensor#torchvision.transforms.ToTensor), 그리고 주어진 평균과 표준편차 값으로 입력값을 normalize 하는 [Normalize()](https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=normalize#torchvision.transforms.Normalize)가 포함되어 있습니다.

여기서 각각의 데이터 증식 기법에 대한 설명은 다음과 같습니다.
- [transforms.RandomRotation](https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=randomrotation#torchvision.transforms.RandomRotation): 주어진 범위 사이의 각도 중에서 무작위로 이미지를 회전시킵니다.
- [transforms.RandomHorizontalFlip](https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=randomhorizontalflip#torchvision.transforms.RandomHorizontalFlip): 주어진 확률로 이미지를 좌우 반전시킵니다.
- [transforms.RandomResizedCrop](https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=randomresizedcrop#torchvision.transforms.RandomResizedCrop): 주어진 scale과 ratio를 통해 이미지 크기를 조정하고 잘라낸 후에, 주어진 이미지 크기로 다시 resize 해줍니다. 우리가 가진 원본 데이터는 이미지마다 크기가 다르기 때문에 모델에 입력으로 주기 위해서는 반드시 일정한 크기로 맞추어 줘야 합니다. 우리 모델에서는 120x120로 입력 이미지의 크기를 고정하겠습니다.

각각의 기법을 적용한 예시는 다음과 같습니다.
<img src="../img/cat_augment.png" width="75%" height="75%">

훈련용 데이터에 대해 각가 기법이 차례대로 적용된 후, ToTensor()와 Normalize()가 적용됩니다. 이번 실습에서 normalization의 평균과 표준편차는 우리가 가진 데이터셋 이미지의 RGB 채널별 평균과 표준편차를 사용합니다. 하지만 이 작업이 번거로운 경우에는 아래에 하드코딩한 평균과 표준편차를 사용하기도 합니다. 이 평균과 표준편차는 대규모 이미지 데이터셋인 [ImageNet](http://www.image-net.org/)의 평균과 표준편차입니다. 

검증 또는 테스트를 진행하는 경우에는, 실행할 때마다 일관된 결과가 나오도록 하기 위해 데이터 증식 기법을 적용하지 않습니다. 하지만 [trasnforms.Resize](https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=resize#torchvision.transforms.Resize)을 통해 마찬가지로 120x120으로 이미지 크기를 고정해주도록 합니다. 

이렇게 정의한 **data_transforms**를 기반으로 학습용, 검증용, 테스트용 Dataset과 DataLoader를 할당합니다. 

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(120, scale=(0.96, 1.0), ratio=(0.95, 1.05)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize([120, 120]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_data = CatDogDataset(data_dir='../data/my_cat_dog', mode='train', transform=data_transforms['train'])
val_data = CatDogDataset(data_dir='../data/my_cat_dog', mode='val', transform=data_transforms['val'])
test_data = CatDogDataset(data_dir='../data/my_cat_dog', mode='test', transform=data_transforms['val'])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=True)

## 5. 네트워크 설계
이제 학습할 뉴럴네트워크를 설계합니다. 일반적으로 CNN 기반의 분류기는 먼저 일련의 convolution layer를 통해 이미지에서 특징을 추출하고, 마지막에 추출된 특징을 Fully Connected layer를 통해 분류하는 구조를 많이 사용합니다.

우리는 총 4개의 convolution layer를 통해 입력 이미지에서 특징을 추출할 것입니다. 다음을 읽고 코드를 완성해보세요.
- **self.conv** 변수에 일련의 convolution layer를 쌓을 것입니다. Pytorch에서는 [nn.Conv2d](https://pytorch.org/docs/stable/nn.html?highlight=conv2d#torch.nn.Conv2d)를 이용해 convolution layer를 사용할 수 있습니다.
- 먼저 쌓을 convolution layer의 입력 채널 수는 3입니다. 입력 이미지가 RGB 3채널이기 때문입니다. 출력 채널 수는 우리가 자유롭게 정해줄 수 있습니다. 이번에는 출력 채널 수를 32로 하겠습니다.
- 모든 convolution layer의 필터 크기는 3x3으로 하겠습니다. 
- 모든 convolution layer 뒤에는 batch normalization([nn.BatchNorm2d](https://pytorch.org/docs/stable/nn.html?highlight=batchnorm#torch.nn.BatchNorm2d))을 적용하고 비선형 activation function으로 ReLU([nn.ReLU](https://pytorch.org/docs/stable/nn.html?highlight=relu#torch.nn.ReLU))를 사용하겠습니다. ReLU 적용 이후에는 2x2의 maxpooling([nn.Maxpool2d](https://pytorch.org/docs/stable/nn.html?highlight=maxpool#torch.nn.MaxPool2d))을 적용하겠습니다. 
- 두 번째 convolution layer의 입력 채널 수는 32, 출력 채널 수는 64로 합니다.
- 세 번째 convolution layer의 입력 채널 수는 64, 출력 채널 수는 128로 합니다.
- 마지막 convolution layer의 입력 채널 수는 128, 출력 채널 수는 128로 합니다.

이번엔 분류기 역할을 하는 fully connected layer를 정의할 차례입니다. 다음을 읽고 코드를 완성해보세요.
- 우리는 총 2개의 fc layer를 쌓을 것입니다. 
- 첫 번째 fc layer(**self.fc1**)의 입력 뉴런 수는 convolution layer에서 가장 마지막에 추출된 특징의 크기에 따라 정해야 합니다. 여러분이 위의 설명대로 convolution layer를 쌓았다면 가장 마지막에 추출되는 특징의 크기는 128x5x5(채널x높이x너비)입니다. 첫 번째 fc layer의 출력 뉴런 수는 512로 해줍니다.
- 마지막 fc layer(**self.fc2**의 출력 뉴런 수는 분류하고자 하는 class 개수와 같아야 합니다. 

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            # 코드 시작
            
            # 코드 종료
        )
        # 코드 시작
        # self.fc1 = 
        # self.fc2 = 
        # 코드 종료
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## 6. train, validation, test 함수 정의
이번에는 훈련, 검증, 테스트를 진행하는 함수를 정의하겠습니다. 

먼저 훈련 함수입니다. 다음을 읽고 코드를 완성해 보세요.
- **imgs**와 **labels**를 **device**로 보내세요.
- **train** 함수에 여러 인자들이 보입니다. **model**이 우리가 선언한 모델일 때, 모델에 입력 이미지를 주고 얻은 결과를 **outputs**에 저장합니다.
- **criterion**이 우리가 선언한 loss function일 때, **outputs**와 **labels**를 통해 loss를 계산하고 그 결과를 **loss**에 저장합니다.
- **optim**이 우리가 선언한 optimizer일 때, 이전에 계산한 기울기를 모두 clear하고, backpropagation을 통해 기울기를 계산하고, optimizer를 통해 파라미터를 업데이트합니다. 

**tarin** 일정한 에폭마다 다음에 구현할 **validation**함수를 통해 검증을 수행합니다. 모델 검증을 수행했을 때, 만약 검증 과정의 평균 loss가 현재까지 가장 낮다면 가장 잘 훈련된 모델로 가정하고 그때까지 학습한 모델을 저장합니다. 저장은 추후에 구현할 **save_model** 함수가 수행합니다. 

In [ ]:
def train(num_epochs, model, data_loader, criterion, optim, saved_dir, val_every, device):
    print('Start training..')
    best_loss = 9999999
    for epoch in range(num_epochs):
        for i, (imgs, labels) in enumerate(data_loader):
            # 코드 시작
            # imgs = 
            # labels = 
            # outputs = 
            # loss = 

            # Clear gradients
            # Gradients 계산
            # Parameters 업데이트
            # 코드 종료

            _, argmax = torch.max(outputs, 1)
            accuracy = (labels.cpu() == argmax.cpu()).float().mean()

            if (i+1) % 1 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                    epoch+1, num_epochs, i+1, len(data_loader), loss.item(), accuracy.item() * 100))

        if (epoch + 1) % val_every == 0:
            avrg_loss = validation(epoch + 1, model, val_loader, criterion, device)
            if avrg_loss < best_loss:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_model(model, saved_dir)


검증 함수입니다. 다음을 읽고 코드를 완성해보세요.
- **imgs**와 **labels**를 **device**로 보내세요.
- 검증 과정에서는 파라미터 업데이트를 하지 않기 때문에 기울기를 계산할 필요는 없습니다. 하지만 검증 과정에서의 평균 loss를 계산하기 위해 loss는 계산해야 합니다. 
- **train** 함수와 마찬가지로 **model**에 입력 이미지를 주어 얻은 결과를 **outputs**에 저장하고, **criterion**을 통해 loss를 계산한 뒤, 그 결과를 **loss**에 저장합니다.

모델 검증 과정에서는 [model.eval()](https://pytorch.org/docs/stable/nn.html?highlight=eval#torch.nn.Module.eval)을 통해 모델을 evaluation 모드로 작동해줘야 함을 기억하시기 바랍니다. Batch normalization 과 Dropout은 훈련과 검증시에 작동하는 방식이 다르기 때문입니다. 평가가 끝난 후에는 다시 [model.train()](https://pytorch.org/docs/stable/nn.html?highlight=module%20train#torch.nn.Module.train)을 통해 train 모드로 바꿔줘야 하는 사실도 잊지 마세요.

In [ ]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch) )
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        total_loss = 0
        cnt = 0
        for i, (imgs, labels) in enumerate(data_loader):
            # 코드 시작
            # imgs = 
            # labels = 
            # outputs =
            # loss = 
            # 코드 종료
            total += imgs.size(0)
            _, argmax = torch.max(outputs, 1)
            correct += (labels.cpu() == argmax.cpu()).sum().item()
            total_loss += loss
            cnt += 1
        avrg_loss = total_loss / cnt
        print('Validation #{}  Accuracy: {:.2f}%  Average Loss: {:.4f}'.format(epoch, correct / total * 100, avrg_loss))
    model.train()
    return avrg_loss

테스트 함수입니다. 다음을 읽고 코드를 완성해보세요.
- **imgs**와 **labels**를 **device**로 보내세요.
- **model**에 입력 이미지를 주어 얻은 결과를 **outputs**에 저장하세요.

테스트에서는 loss를 계산할 필요가 없고, 전체 정확도를 통해 모델의 성능을 확인하면 됩니다. 

In [ ]:
def test(model, data_loader, device):
    print('Start test..')
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (imgs, labels) in enumerate(test_loader):
            # 코드 시작
            # imgs = 
            # labels = 
            # outputs = 
            # 코드 종료
            _, argmax = torch.max(outputs, 1) # max()를 통해 최종 출력이 가장 높은 class 선택
            total += imgs.size(0)
            correct += (labels.cpu() == argmax.cpu()).sum().item()

        print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total * 100))
    model.train()

## 7. 모델 저장 함수 정의
모델을 저장하는 함수입니다. 모델 저장은 [torch.save](https://pytorch.org/docs/stable/torch.html?highlight=save#torch.save) 함수를 통해 할 수 있습니다. 
[nn.Module.state_dict](https://pytorch.org/docs/stable/nn.html?highlight=state_dict#torch.nn.Module.state_dict)를 통해 Module, 즉 우리 모델의 파라미터를 가져올 수 있습니다. 이렇게 불러온 파라미터를 **check_point** 딕셔너리에 저장합니다. 그리고 이 **check_point**를 정해준 경로에 저장하면 됩니다. 

torch.save는 단순히 모델의 파라미터만 저장하는 함수가 아닙니다. 어떤 파이썬 객체든 저장할 수 있습니다. 그래서 경우에 따라 **check_point** 딕셔너리에 모델의 파라미터 뿐만 아니라 다른 여러 가지 필요한 정보를 저장할 수도 있습니다. 예를 들어 총 몇 에폭동안 학습한 모델인지 그 정보도 저장할 수 있겠죠? 

다음을 읽고 코드를 완성해보세요.
- torch.save를 통해 **output_path** 경로에 **check_point**를 저장하세요.

In [ ]:
def save_model(model, saved_dir, file_name='best_model.pt'):
    os.makedirs(saved_dir, exist_ok=True)
    check_point = {
        'net': model.state_dict()
    }
    output_path = os.path.join(saved_dir, file_name)
    # 코드 시작

    # 코드 종료

## 8. 모델 생성 및 Loss function, Optimizer 정의
이제 학습할 모델을 생성하고 loss function과 optimizer를 정의할 차례입니다. 다음을 읽고 코드를 완성해보세요.
- 위에서 정의한 SimpleCNN class를 통해 모델을 생성하고 이를 **model** 변수에 저장합니다.
- 분류 문제에서는 cross entropy loss를 사용합니다. cross entropy loss function([nn.CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html?highlight=crossentropy#torch.nn.CrossEntropyLoss))을 만들고 **criterion** 변수에 저장합니다.
- 이번 실습에서는 Adam optimizer를 통해 파라미터를 업데이트 하겠습니다. Adam optimizer([torch.optim.Adam](https://pytorch.org/docs/stable/optim.html?highlight=adam#torch.optim.Adam))를 **optimizer** 변수에 저장합니다.

**val_every**는 검증을 몇 에폭마다 진행할지 정하는 변수입니다. **saved_dir**은 모델이 저장될 디렉토리의 경로입니다. 

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# 코드 시작
model = 
criterion = 
optimizer = 
# 코드 종료
val_every = 1
saved_dir = '../saved/SimpleCNN'

## 9. Training
**train** 함수를 통해 학습을 진행합니다. 네트워크의 규모가 큰 편이 아니지만, CPU를 통해 학습되기 때문에 시간이 조금 필요합니다. 컴퓨터 성능에 따라 20~30분의 시간이 소요될 수 있습니다. 시간이 여유가 없는 분들은 모델 학습이 적당히 진행된다는 정도만 확인하고 다음 단계로 넘어가셔도 됩니다. 

만약 어느정도 기다렸음에도 학습 accuracy가 증가하지 않는다면 구현한 코드에 문제가 있을 수 있습니다. 이러한 경우에는 구현한 train 함수를 다시 한 번 확인하시기 바랍니다. 

또한, 모델 저장 코드를 제대로 구현했다면 첫 에폭 학습후에 ../saved/SimpleCNN 경로에 best_model.pt 파일이 저장되어 있어야 합니다. 만약에 파일이 존재하지 않는다면 모델 저장 코드를 다시 확인하시기 바랍니다.

In [ ]:
train(num_epochs, model, train_loader, criterion, optimizer, saved_dir, val_every, device)

## 10. 저장된 모델 불러오기 및 test
학습한 모델의 성능을 테스트합니다. 저장한 모델 파일을 [torch.load](https://pytorch.org/docs/stable/torch.html?highlight=load#torch.load)를 통해 불러옵니다. 위에서 학습을 끝까지 진행하지 않았다면, 아래의 주석 처리된 부분을 주석 해제하면, 제공해드린 미리 학습시킨 모델을 불러올 수 있습니다. 

이렇게 불러오면 우리가 얻게 되는 건 아까 저장한 **check_point** 딕셔너리입니다. 딕셔너리에 저장한 모델의 파라미터는 **'net'** key에 저장해두었습니다. 이를 불러와 **state_dict**에 저장합니다. 이렇게 불러온 모델의 파라미터를 모델에 실제로 로드하기 위해서는 [nn.Module.load_state_dict](https://pytorch.org/docs/stable/torch.html?highlight=load#torch.load)를 사용하면 됩니다. 

다음을 읽고 코드를 완성해보세요.
- **model_path**의 경로에 있는 모델 파일을 로드하여, 이를 **check_point** 변수에 저장합니다. 
- **check_point** 딕셔너리에 접근하여 모델의 파라미터를 **state_dict** 변수에 저장합니다.
- **state_dict**의 파라미터들을 우리 모델에 로드합니다.

In [ ]:
model_path = '../saved/SimpleCNN/best_model.pt'
model = SimpleCNN().to(device) # 아래의 모델 불러오기를 정확히 구현했는지 확인하기 위해 새로 모델을 선언하여 학습 이전 상태로 초기화
# 코드 시작
# checkpoint = 
# state_dict = 

# 코드 종료
val_every = 1
saved_dir = '../saved/SimpleCNN'

마지막으로 모델의 성능을 테스트합니다. 76% 내외의 성능이 나온다면 학습 및 모델 불러오기가 성공적으로 진행된 것입니다. 

In [ ]:
test(model, test_loader, device)

## 11. VGG

<img src="../img/vgg16.png" width="15%" height="15%">

In [ ]:
class VGG16(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.conv = nn.Sequential(
            # 코드 시작
            
            # 코드 종료
        )
        # 코드 시작
        # self.fc1 = 
        # 코드 종료
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = VGG16().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train(num_epochs, model, train_loader, criterion, optimizer, saved_dir, val_every, device)

In [ ]:
test(model, test_loader, device)

## 12. Transfer Learning
실습을 끝내기전에, 우리는 전이 학습(transfer learning)을 구현해 보고 성능을 확인해 볼 것입니다. 
전이학습이란 비슷한 목적으로 미리 학습된 모델의 파라미터로 나의 모델의 파라미터를 초기화한 후 학습을 이어서 진행하는 것을 말합니다. 그렇다면 이러한 전이학습은 어떤 장점이 있기에 사용하는 것일까요?

여러분이 현실에서 딥러닝을 활용할 때 흔히 마주칠 수 있는 현실적인 제약들이 있습니다. 데이터 부족, 컴퓨팅 리소스 부족, 시간의 부족이 대표적인 현실적인 제약들에 속합니다. 우리가 풀고자 하는 문제와 완전히 똑같지는 않지만 어느정도 연관성이 있는 문제를, 아주 많은 양의 데이터로, 미리 학습한 모델이 있다면 그 모델은 정말 아무것도 모르는 백지 상태의 모델보다 우리가 풀고 싶은 문제에 대해 훨씬 더 빨리 배우고 잘 배울 수 있습니다. 

구현은 전혀 어렵지 않습니다. 먼저 미리 학습된 모델을 불러와야 합니다. 
여기서 불러올 모델은 ResNet으로, 대규모 ImageNet 데이터로 이미지 분류를 학습한 모델입니다.

torchvision에 구현된 resnet50을 파라미터가 학습된 상태로 불러옵니다. 학습된 파라미터를 다운받기 위해 몇 분의 시간이 소요될 수 있습니다.

In [ ]:
new_model = torchvision.models.resnet50(pretrained=True)

다만 우리가 앞서 직접 정의한 모델(SimpleCNN)에서는 입력 이미지의 크기를 120x120로 한 것에 반해, 방금 불러온 ResNet은 입력 이미지 크기를 최소한 224x224로 가정하고 학습된 모델입니다. 따라서 입력 데이터 크기만 수정하여 DataLoader를 다시 정의하도록 하겠습니다. 

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224, scale=(0.96, 1.0), ratio=(0.95, 1.05)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_data = CatDogDataset(data_dir='../data/my_cat_dog', mode='train', transform=data_transforms['train'])
val_data = CatDogDataset(data_dir='../data/my_cat_dog', mode='val', transform=data_transforms['val'])
test_data = CatDogDataset(data_dir='../data/my_cat_dog', mode='test', transform=data_transforms['val'])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=True)

우리가 불러온 ImageNet에 학습된 ResNet은 1000개의 class를 구분하는 네트워크입니다. 즉, 마지막 FC layer의 출력 뉴런 수가 1000개인 것입니다. 우리는 2개의 class를 구분하는 네트워크를 원하기 때문에, 마지막 FC layer만 출력을 2로 바꿔주고 이 부분에 대해서만 학습을 추가로 진행하면 됩니다.

상황에 따라서는 우리가 이번에 하는 것처럼 마지막 FC layer만 학습을 진행하는 것이 아니라 전체 네트워크에 대해서 학습을 이어서 진행하는 경우도 있습니다. 이를 파라미터 fine tuning(미세 조정)이라고 합니다. ImageNet에는 다양한 동물 class도 포함이 되어있습니다. 즉, 우리가 불러온 ResNet은 강아지와 고양이 같은 동물에 대한 특징을 이미 어느정도 잘 추출하는 네트워크인 것입니다. 따라서 fine tuning이 굳이 필요하지 않습니다. 게다가, 이번 실습과 같이 적은 양의 데이터를 통해 모델을 학습시키는 상황에서 fine tuning을 진행하는 것은 overfitting의 가능성이 커지는 것이기 때문에 오히려 성능을 낮추는 결과를 가져올 수 있습니다. 

이러한 내용을 아래 코드에 구현해두었습니다.
- [nn.Module.parameters](https://pytorch.org/docs/stable/nn.html?highlight=parameters#torch.nn.Module.parameters)를 통해 모델 파라미터에 대한 iterator를 가져올 수 있습니다. 먼저, 이 iterator를 통해 for문 을 돌며 모든 파라미터에 대해 [requires_grad](https://pytorch.org/docs/stable/autograd.html?highlight=requires_grad#torch.Tensor.requires_grad)를 False로 바꿔줍니다. 이렇게 하면 이 파라미터들에 대해서는 기울기가 계산되지 않기 때문에, 파라미터 업데이트가 되지 않습니다.
- 그 다음으로는 맨마지막 FC layer를 새로 정의해주는 것입니다. 그런데 문제는 마지막 FC layer에 어떻게 접근하느냐 입니다. 우리가 구현을 하지 않았기 때문에 마지막 layer의 입력 뉴런수가 어떻게 되는지도 모르는 상황입니다. 그러면 우리가 할 일은 구현된 코드를 보는 것입니다. torchvision에 우리가 불러온 ResNet이 구현되어 있습니다. [이곳](https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py#L146)에 가면 구현된 ResNet의 마지막 레이어의 변수명이 **self.fc**로 되어있음을 알 수 있습니다. 따라서 우리는 **모델.fc**와 같은 방식으로 이 layer에 접근할 수 있습니다. 이러한 방식으로 마지막 FC layer의 입력 뉴런수를 가져오고, 출력 뉴런 수는 우리의 문제에 맞게 2로 하여 마지막 layer를 수정할 수 있습니다. 

다음을 읽고 코드를 완성해보세요.
- 불러온 모델의 가장 마지막 FC layer의 이름은 'fc'임을 확인했습니다. 해당 레이어의 입력 뉴런 수를 **num_ftrs** 변수에 저장했습니다. 이제, 이 마지막 레이어에 새로운 FC layer를 정의하고, 그것의 입력 뉴런 수는 이전과 마찬가지로 **num_ftrs**로 하고 출력 뉴런 수는 우리가 분류하고자 하는 class의 총 개수로 해주세요,
- 우리의 새로운 모델 학습에 사용될 loss function과 optimizer를 다시 정의합니다. 
- Cross entorpy loss function을 선언하고 이를 criterion 변수에 담습니다.
- Adam optimizer를 선언하고 이를 opitmizer 변수에 담습니다. 인자로 넣어 주는 모델 파라미터는 새로 불러온 모델의 파라미터여야 함에 주의하세요. learning rate는 이전과 똑같이 지정합니다. 

In [ ]:
for param in new_model.parameters():
    param.requires_grad = False

num_ftrs = new_model.fc.in_features
new_model.fc = nn.Linear(num_ftrs, 2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(new_model.parameters(), lr=learning_rate)
val_every = 1
saved_dir = '../saved/ResNet50'

불러온 모델에 대해 학습을 진행합니다. 이 과정도 100분 내외의 시간이 소요됩니다. 마찬가지로 적당히 학습이 진행되는 것만 보고 중단후 다음 단계로 넘어가셔도 됩니다.

In [ ]:
train(num_epochs, new_model, train_loader, criterion, optimizer, saved_dir, val_every, device)

전이 학습을 마친 모델을 불러오고, 그 성능을 확인합니다. 마찬가지로 위에서 학습을 끝까지 진행하지 않았다면 아래 주석부분을 해제하시기 바랍니다.

다음을 읽고 코드를 완성해보세요.
- **model_path**의 경로에 있는 모델 파일을 로드하여, 이를 **check_point** 변수에 저장합니다. 
- **check_point** 딕셔너리에 접근하여 모델의 파라미터를 **state_dict** 변수에 저장합니다.
- **state_dict**의 파라미터들을 모델에 로드합니다.

In [ ]:
model_path = '../saved/ResNet50/best_model.pt'
checkpoint = torch.load(model_path)
state_dict = checkpoint['net']
new_model.load_state_dict(state_dict)

테스트를 수행합니다. 정확도가 91% 내외가 나온다면 성공적으로 진행된 것입니다. 겨우 마지막 FC layer만 학습시켰음에도 불구하고 우리의 SimpleCNN보다 성능이 훨씬 좋은 것을 볼 수 있습니다.

In [ ]:
test(new_model, test_loader, device)

## 12. Summary

우리는 이번 실습을 통해 다음과 같은 내용을 학습했습니다.
- Dataset class를 우리가 가진 데이터셋에 맞게 customize 하여 정의할 수 있다.
- CNN을 설계하고 이미지 분류기를 학습시킬 수 있다.
- 학습된 모델을 저장하고 불러올 수 있다.
- 데이터, 리소스, 시간이 부족한 상황에서 전이학습을 사용하여 이를 극복할 수 있다.